### Histórico de Voos

O [Voo Regular Ativo – VRA](http://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/historico-de-voos)  é uma base de dados composta por informações de voos de empresas de transporte aéreo que apresenta os cancelamentos e horários em que os voos ocorreram.

A ANAC torna pública a série histórica do Voo Regular Ativo para que sejam realizados estudos e análises. 

A base de dados mensal do VRA, é disponibilizada em formato “csv”. 
O mês é em referência às etapas cujas decolagens eram previstas para o mês em questão ou cujas decolagens, em caso de etapa não prevista, foram realizadas no mês em questão.

Utilizando a técnica de Web Scrapping, iremos buscar todos os links para download dos respectivos arquivos de interesse.

###### Importante
Foi necessário utilização de Web Scrapping pois os arquivos não possuem padronização de nomenclatura, pois alguns estão como vra e outros VRA.

**Index**
1. [Coleta de Dados (Web Scrapping)](#Coleta-de-Dados-(Web-Scrapping))
2. [Download Arquivos](#Download-Arquivos)

### Coleta de Dados (Web Scraping)

In [1]:
import requests as rq
from bs4 import BeautifulSoup

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
}

url = "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/historico-de-voos"
req = rq.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')

# Obtem os links para os arquivos
url_arquivos = soup.find_all(class_="internal-link")
links = [url.attrs['href'] for url in url_arquivos]

In [2]:
# Filtro para os links de interesses

anos = ['2022']

links_arquivos = []
for ano in anos:
    links_arquivos.append([link for link in links if ano in link])

### Download Arquivos

In [3]:
import os
import time
import chardet
import pandas as pd
from io import StringIO

path = f"dados/historico_ANAC/"
dados_compilados = []

if not os.path.exists(path):
    print("Criado caminho:", path)
    os.makedirs(path)


for url in links_arquivos[0]:
    print(url)
    response = rq.get(url)
    nomeArquivo = url.split('/')[-1]
    
    with open("dados/historico_ANAC/{}".format(nomeArquivo),"wb")as file:
        bytes_data = response.content
        # file.write(data)
        
        # Verificação da codificacao do arquivo
        result_encoding = chardet.detect(bytes_data[:100])
        s=str(bytes_data, result_encoding['encoding'])
        data = StringIO(s) 

        df_tmp = pd.read_csv(data, sep=';', low_memory=False)
        dados_compilados.append(df_tmp)
        

Criado caminho: dados/historico_ANAC/
https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2022/vra_01_2022.csv
https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2022/VRA_02_2022.csv
https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2022/VRA_03_2022.csv
https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2022/vra_04_2022.csv
https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2022/VRA_05_2022.csv
https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2022/vra_06_2022.csv
https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2022/vra_07_2022.csv
https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2022/VR

In [4]:
df_voos = pd.concat(dados_compilados)

In [5]:
df_voos['Código DI'] = df_voos['Código DI'].astype(str)

In [6]:
df_voos.to_parquet('df_voos.parquet')